In [ ]:
! pip install --upgrade google-cloud-aiplatform google-generativeai langchain langchain-google-genai
! pip install "langchain[docarray]"

## Restart session

In [ ]:
import IPython
from IPython.display import display

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# Define project information
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}


import vertexai
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)
import langchain
from langchain.llms import VertexAI
from google.cloud import aiplatform


# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

In [ ]:
model = GenerativeModel("gemini-pro")

responses = model.generate_content("Tell me a funny joke about dogs?",
                                   stream=True)

for response in responses:
    print(response.text, end="")

## Input an API Key

Create this in you project under APIs and Services | Credentials

In [ ]:
import getpass
import os

# getpass will prompt for an API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)
result = llm.invoke("Tell me a funny joke about dogs")

print(result.content)

## A Simple Chain

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"topic": "space aliens"})

## RAG with Embeddings

This uses an in-memory Vector Store.

In [ ]:
! pip install "langchain[docarray]"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Vector Store takes array of text plpus the embeddings model
# as parameters
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Why did the cat get kicked out of the restaurant? Because it was not well-mannered.",
     "Why did the dog go to the park? To chase squirrels and sniff butts!",
     "Why did the COBOL programmer get lost in the forest? Because he didn’t know how to exit.",
     "Why did the computer science student get a geometry textbook? To improve his algorithms.",
     "Why did the basketball player bring a belt to the game? To hold up his shorts!",
     "Why did the aliens get sent to the principal's office? Because they were from a different planet."
     ],

    embedding=embeddings # passing in the embedder model
)

retriever = vectorstore.as_retriever()


In [ ]:
docs = retriever.get_relevant_documents("Joke about cats", )
print(docs[0].page_content)

In [ ]:
docs = retriever.get_relevant_documents("Joke about programming", )
print(docs[0].page_content)

In [ ]:
docs = retriever.get_relevant_documents("Tell me a Joke about dogs", )
print(docs[0].page_content)

## Multi-Modal

In [ ]:
import requests
from IPython.display import Image

image_url = "https://storage.googleapis.com/doug-rehnstrom-public/newyork.JPG"

content = requests.get(image_url).content
Image(content,width=600)



In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")

# example
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What is the location of this picture? Are there and famous landmarks in the picture? How many people live there?",
        },  # You can optionally provide text parts
        {
            "type": "image_url",
            "image_url": image_url
         },
    ]
)

llm.invoke([message])